In [1]:
import pandas as pd
import numpy as np

In [10]:
pd.read_csv('data_set_20210520_v_0_01.csv', parse_dates=['Date_Start','Date_End'])

,Vessel_Name,Date_Start,Date_End,Vessel_Position_Latitude,Vessel_Position_Longitude,Vessel_Speed,Deadweigth_tonage,Main_Engine_Total_Revolution,Main_Engine_Working_Hours,Main_Engine_Total_Consumption
0,Vessel_1,2019-08-02 03:00:00,2019-08-03 02:00:00,36.813800,135.224816,0.0000,23767.956,NaN,NaN,NaN
1,Vessel_1,2019-08-03 02:00:00,2019-08-04 09:12:00,36.525833,134.891334,NaN,23767.956,34304.000,10.0104,8.0446
2,Vessel_1,2019-08-04 09:12:00,2019-08-05 07:30:00,37.488667,138.743334,3.1598,35448.732,76297.456,23.0136,18.9544
3,Vessel_1,2019-08-05 07:30:00,2019-08-06 03:00:00,37.630000,138.886000,NaN,35448.732,5664.448,1.7544,1.2122
4,Vessel_1,2019-08-06 03:00:00,2019-08-07 03:00:00,37.612333,138.850334,NaN,35448.732,4263.344,1.4448,0.7714
...,...,...,...,...,...,...,...,...,...,...
31946,Vessel_68,2020-01-05 16:00:00,2020-01-06 16:00:00,14.946000,-65.287834,0.0000,72469.656,NaN,NaN,NaN
31947,Vessel_68,2020-01-06 16:00:00,2020-01-07 16:00:00,14.857667,-65.287834,0.0000,72469.656,NaN,NaN,NaN
31948,Vessel_68,2020-01-07 16:00:00,2020-01-07 22:42:00,14.850600,-65.287834,0.0000,72469.656,NaN,NaN,NaN
31949,Vessel_68,2020-01-07 22:42:00,2020-01-08 02:00:00,14.822333,-65.287834,NaN,72469.656,4132.560,1.5480,0.4959
